In [1]:
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys 
import os 
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import matplotlib.pyplot as plt


In [30]:
# 크롬 드라이버를 켜주는 옵션
chrome_options = Options()
driver = webdriver.Chrome(executable_path='./chromedriver',options=chrome_options)

C:\Users\KHJ\AppData\Local\Temp\ipykernel_8424\3746816502.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver',options=chrome_options)


In [378]:
driver.get("https://kr.trip.com/travel-guide/") # trip.com 접속
keyword = '캠핑 대한민국' # 다국적으로 결과가 나오기에 나라 이름과 캠핑을 같이 입력
Keyword=driver.find_element_by_xpath('//*[@id="__next"]/div[2]/div/div[1]/div[3]/div/div[1]/input') # 검색창 소스 코드
Keyword.send_keys(keyword) 
driver.find_element_by_xpath('//*[@id="__next"]/div[2]/div/div[1]/div[3]/div/div[2]/div').click() # 검색 클릭


time.sleep(3) # 결과가 나오기까지 여유 시간

In [379]:
driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[3]/div/ul/li[2]').click() # 캠핑장 분류중 위의 것을 클릭
time.sleep(3)

In [380]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') # 파싱

camp = soup.select('div.content')

In [381]:

camping_list = []
for camping in camp:
    name = str(camping.select('div > a')[0]).split('title="')[1].split('"><img')[0] # 파싱한 결과물에서 소스코드로 캠핑장 이름 출력
    page = str(camping.select('div.gl-search-result_list-title > a')[0]).split('a href="')[1].split('" title=')[0] # 리뷰를 위한 페이지 추출
    camping_list.append([name,page]) # 빈 리스트에 결과 저장
camping_list

[['JSKI',
  'https://kr.trip.com/travel-guide/attraction/hongcheon-gun/jski-55990764/'],
 ['제이스키',
  'https://kr.trip.com/travel-guide/attraction/pyeongchang-gun/jski-55986626/'],
 ['곡성군청소년야영장',
  'https://kr.trip.com/travel-guide/attraction/gokseong-gun/gokseong-gun-youth-camp-65176640/'],
 ['코리아인터내셔널스키스쿨',
  'https://kr.trip.com/travel-guide/attraction/hongcheon-gun/korea-international-ski-school-99590933/'],
 ['Boseong',
  'https://kr.trip.com/travel-guide/attraction/boseong-gun/boseong-61170193/']]

In [382]:
columns = ['이름', '페이지']
pd_data = pd.DataFrame(camping_list, columns = columns) # 데이터프레임으로 저장
pd_data.head()


,이름,페이지
0,JSKI,https://kr.trip.com/travel-guide/attraction/ho...
1,제이스키,https://kr.trip.com/travel-guide/attraction/py...
2,곡성군청소년야영장,https://kr.trip.com/travel-guide/attraction/go...
3,코리아인터내셔널스키스쿨,https://kr.trip.com/travel-guide/attraction/ho...
4,Boseong,https://kr.trip.com/travel-guide/attraction/bo...


In [383]:
for i in range(len(camping_list)):
    driver.get(pd_data['페이지'][i]) # 데이터프레임에서 페이지 주소를 꺼내옴
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    nearname = str(soup.select('div.gl-map_list-content_name.show_one_line > span')).split('</span')[0].split('span>')[1] # 인근 명소 추출
    address = soup.select('div.address > div > div > div > div > span.field')[0].text # 주소 추출
    review_data = []

    reviews = soup.select('div.gl-poi-detail_comment-content > p') # 리뷰 추출
    for review in reviews:
        review = str(review).strip('<p>').strip('</')
        review_data.append(review)

    review_data = ",".join(review_data) # 리뷰들을 하나로 엮어줌
    camping_list[i].extend([nearname,address,review_data]) # 리스트에 근처 명소와 주소, 리뷰 데이터까지 합쳐줌

In [443]:
driver.get("https://kr.trip.com/travel-guide/")
keyword = '캠핑 대한민국'
Keyword=driver.find_element_by_xpath('//*[@id="__next"]/div[2]/div/div[1]/div[3]/div/div[1]/input')
Keyword.send_keys(keyword) 
driver.find_element_by_xpath('//*[@id="__next"]/div[2]/div/div[1]/div[3]/div/div[2]/div').click()
time.sleep(3)
driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[3]/div/ul/li[3]').click() # trip.com의 호텔에 해당하는 캠핑장 클릭
time.sleep(3)

In [444]:
camping_list2 = []
for page in range(1, 20):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    camp = soup.select('div.content')
    for camping in camp:
        name = str(camping.select('div > a')[0]).split('title="')[1].split('"><img')[0]
        page = str(camping.select('div.gl-search-result_list-title > a')[0]).split('a href="')[1].split('" title=')[0]
        camping_list2.append([name,page])

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # 스크롤을 내리는 소스 코드 입력
    try :
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[3]/div/div/div[4]/div/button[2]').click()
        # 다음 번 페이지로 넘어가는 소스코드를 잡아준 후 클릭 명령 
        time.sleep(3)
    except:
        pass


camping_list2

[['고성 쭌 캠핑펜션',
  'https://kr.trip.com/hotels/cityname-hotel-detail-75873831/hotelname/'],
 ['냇강마을산촌황토펜션&amp;캠핑',
  'https://kr.trip.com/hotels/cityname-hotel-detail-67271660/hotelname/'],
 ['주문진 글램핑 오토캠핑장',
  'https://kr.trip.com/hotels/cityname-hotel-detail-45587711/hotelname/'],
 ['홍천 비발디캠핑파크카라반',
  'https://kr.trip.com/hotels/cityname-hotel-detail-71652880/hotelname/'],
 ['카라반베이 캠핑 - 카라반 파크',
  'https://kr.trip.com/hotels/cityname-hotel-detail-8503289/hotelname/'],
 ['가평 운악홀리데이 카라반캠핑장',
  'https://kr.trip.com/hotels/cityname-hotel-detail-76194217/hotelname/'],
 ['서해바다 글램핑 - 캠핑장"><div class="lazyload-placeholder"></div></a>',
  'https://kr.trip.com/hotels/cityname-hotel-detail-21993277/hotelname/'],
 ['에코힐 글램핑 제주 - 캠핑장"><div class="lazyload-placeholder"></div></a>',
  'https://kr.trip.com/hotels/cityname-hotel-detail-7612480/hotelname/'],
 ['무지개 펜션캠핑 인천"><div class="lazyload-placeholder"></div></a>',
  'https://kr.trip.com/hotels/cityname-hotel-detail-9131720/hotelname/'],
 ['대부도 캠핑성

In [445]:
columns = ['이름', '페이지']
pd_data2 = pd.DataFrame(camping_list2, columns = columns)
pd_data2.head()


,이름,페이지
0,고성 쭌 캠핑펜션,https://kr.trip.com/hotels/cityname-hotel-deta...
1,냇강마을산촌황토펜션&amp;캠핑,https://kr.trip.com/hotels/cityname-hotel-deta...
2,주문진 글램핑 오토캠핑장,https://kr.trip.com/hotels/cityname-hotel-deta...
3,홍천 비발디캠핑파크카라반,https://kr.trip.com/hotels/cityname-hotel-deta...
4,카라반베이 캠핑 - 카라반 파크,https://kr.trip.com/hotels/cityname-hotel-deta...


In [448]:
for i in range(len(camping_list2)):
    driver.get(pd_data2['페이지'][i])
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    address = soup.select('span > span.detail-headline_position_text')[0].text    
    
    element = driver.find_element_by_xpath('//*[@id="ibu-hotel-detail-head"]/div[2]/div[2]/div[2]/div[2]/div/div/div[2]')
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)

    nearname = driver.find_element_by_css_selector('p.detail-map-poilist_item_title').text
    review_data = '없음' # 호텔쪽에는 리뷰 데이터가 없었기에 생략됨
    
    camping_list2[i].extend([nearname,address,review_data])

In [449]:
camping_list = camping_list + camping_list2 # trip.com의 캠핑장 데이터 리스트 통합

In [454]:
columns = ['이름','페이지','주변명소', '주소', '리뷰데이터']
pd_data = pd.DataFrame(camping_list, columns = columns)
pd_data

,이름,페이지,주변명소,주소,리뷰데이터
0,JSKI,https://kr.trip.com/travel-guide/attraction/ho...,남이섬,서면 한서로 2217 | 강원도 홍천군 스노우뷰 25103,우리는 JSKI에 대한 좋은 리뷰를 읽은 후 4 명의 가족 (6 명과 2 명)을위한...
1,제이스키,https://kr.trip.com/travel-guide/attraction/py...,용평리조트스키장,대관령면 솔봉로 325 | 강원도 평창군 알펜시아 리조트,우리 여행 3 개월 전에 용평 스키 리조트에서 2 시간 / 4 시간 개인 스키 수업...
2,곡성군청소년야영장,https://kr.trip.com/travel-guide/attraction/go...,순천드라마촬영장,전라남도 구청군 구다면 두쿠리 627-2,
3,코리아인터내셔널스키스쿨,https://kr.trip.com/travel-guide/attraction/ho...,남이섬,강원도 홍천군 서면 한치골길 619,영어 강사로부터 교훈을 배우는 놀라운 경험. 친절하고 친절하며 정시에 전문적입니다....
4,Boseong,https://kr.trip.com/travel-guide/attraction/bo...,순천만자연생태공원,대한민국 전라남도 보성군,
5,고성 쭌 캠핑펜션,https://kr.trip.com/hotels/cityname-hotel-deta...,양양국제공항,대한민국 강원도 고성군 강원 고성군 토성면 화원길 11 24754,없음
6,냇강마을산촌황토펜션&amp;캠핑,https://kr.trip.com/hotels/cityname-hotel-deta...,굿모닝사우나,대한민국 강원도 인제군 북면 냇강마을로 51 24611,없음
7,주문진 글램핑 오토캠핑장,https://kr.trip.com/hotels/cityname-hotel-deta...,양양국제공항,대한민국 강원도 강릉시 주문진읍 신리천로 951-9 25401,없음
8,홍천 비발디캠핑파크카라반,https://kr.trip.com/hotels/cityname-hotel-deta...,원주공항,대한민국 강원도 홍천군 서면 한서로 1045-29 25100,없음
9,카라반베이 캠핑 - 카라반 파크,https://kr.trip.com/hotels/cityname-hotel-deta...,김포국제공항,대한민국 경기도 용인시 포곡읍 전대로 62처인구 17023,없음


In [456]:
pd_data = pd_data.drop_duplicates()
pd_data

,이름,페이지,주변명소,주소,리뷰데이터
0,JSKI,https://kr.trip.com/travel-guide/attraction/ho...,남이섬,서면 한서로 2217 | 강원도 홍천군 스노우뷰 25103,우리는 JSKI에 대한 좋은 리뷰를 읽은 후 4 명의 가족 (6 명과 2 명)을위한...
1,제이스키,https://kr.trip.com/travel-guide/attraction/py...,용평리조트스키장,대관령면 솔봉로 325 | 강원도 평창군 알펜시아 리조트,우리 여행 3 개월 전에 용평 스키 리조트에서 2 시간 / 4 시간 개인 스키 수업...
2,곡성군청소년야영장,https://kr.trip.com/travel-guide/attraction/go...,순천드라마촬영장,전라남도 구청군 구다면 두쿠리 627-2,
3,코리아인터내셔널스키스쿨,https://kr.trip.com/travel-guide/attraction/ho...,남이섬,강원도 홍천군 서면 한치골길 619,영어 강사로부터 교훈을 배우는 놀라운 경험. 친절하고 친절하며 정시에 전문적입니다....
4,Boseong,https://kr.trip.com/travel-guide/attraction/bo...,순천만자연생태공원,대한민국 전라남도 보성군,
5,고성 쭌 캠핑펜션,https://kr.trip.com/hotels/cityname-hotel-deta...,양양국제공항,대한민국 강원도 고성군 강원 고성군 토성면 화원길 11 24754,없음
6,냇강마을산촌황토펜션&amp;캠핑,https://kr.trip.com/hotels/cityname-hotel-deta...,굿모닝사우나,대한민국 강원도 인제군 북면 냇강마을로 51 24611,없음
7,주문진 글램핑 오토캠핑장,https://kr.trip.com/hotels/cityname-hotel-deta...,양양국제공항,대한민국 강원도 강릉시 주문진읍 신리천로 951-9 25401,없음
8,홍천 비발디캠핑파크카라반,https://kr.trip.com/hotels/cityname-hotel-deta...,원주공항,대한민국 강원도 홍천군 서면 한서로 1045-29 25100,없음
9,카라반베이 캠핑 - 카라반 파크,https://kr.trip.com/hotels/cityname-hotel-deta...,김포국제공항,대한민국 경기도 용인시 포곡읍 전대로 62처인구 17023,없음


In [457]:
pd_data.to_csv("travel.csv") # 데이터프레임을 csv 파일로 저장

In [4]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path='./chromedriver',options=chrome_options)

In [5]:
seoul_data = camp_data('서울', 2)

In [ ]:
seoul_review = camp_review_data(seoul_data)

In [ ]:
gyunggi_data.to_csv('gyunggi_camp.csv')

In [13]:
incheon_data = camp_data('인천', 4)

In [ ]:
incheon_review = camp_review_data(incheon_data)

In [9]:
busan_data = camp_data('부산', 2)

In [ ]:
busan_review = camp_review_data(busan_data)

In [51]:
daegu_data = camp_data('대구', 2)

In [ ]:
daegu_review = camp_review_data(deagu_data)

In [19]:
daejun_data = camp_data('대전', 2)

In [ ]:
daejun_review = camp_review_data(daejun_data)

In [20]:
gwangju_data = camp_data('광주', 3)

In [ ]:
gwngju_review = camp_review_data(gwangju_data)

In [19]:
driver.close() # 크롬창 종료

In [27]:
ulsan_data = camp_data('울산', 3)

In [ ]:
ulsan_review = camp_review_data(ulsan_data)

In [33]:
gyunggi_data = camp_data('경기', 5)

In [62]:
def camp_data(location, num): # 지역별 캠핑 데이터를 모으기위한 함수
    driver.get("https://map.naver.com/v5/") # 네이버 지도 진입

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
       ) 
    finally:
        pass
    # 검색창이 나올때까지 대기

    search_box = driver.find_element_by_class_name("input_search") # 검색창 소스 코드
    search_box.send_keys(f"{location} 캠핑장") # location에 해당하는 캠핑장 검색
    search_box.send_keys(Keys.ENTER) 

    time.sleep(7)

    frame = driver.find_element_by_css_selector("iframe#searchIframe") # 프레임 전환을 위한 소스코드

    driver.switch_to.frame(frame) # 프레임 전환
    time.sleep(3)

    scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]") # 스크롤 소스코드
    try:
        for i in range(10): # 스크롤을 맨밑까지 내려줄때까지 반복, 촤대로 내리면 멈추기 위해 try, except 추가
            driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div) 
            time.sleep(2)
    except:
        pass

    loca_list = []
    
    x = 3
    
    try:
        while x <= num+2:  # 2번째 이상 페이지로 넘기기 위한 반복문
            for i in range(50): # 페이지당 캠핑장 숫자
                try:
                    name = driver.find_element_by_xpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i+1}]/div[1]/a/div[1]/div/span').text
                    # 캠핑장 이름 소스코드
                    driver.find_element_by_xpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i+1}]/div[1]/a/div[1]/div/span').click()
                    # 캠핑장 정보를 보기 위한 클릭
                    driver.switch_to.default_content()
                    # 캠핑장 정보 프레임으로 들어가기 위해 현재 프레임을 나옴
                    time.sleep(1)

                    frame_in = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-place-bridge/div/nm-external-frame-bridge/nm-iframe/iframe')
                    # 캠핑장 정보창으로 프레임 전환
                    driver.switch_to.frame(frame_in)
                    address = driver.find_element_by_css_selector('span._2yqUQ').text
                    # 캠핑장 주소 소스코드
                    try:
                        phone = driver.find_element_by_css_selector('span._3ZA0S').text # 전화번호 소스코드
                    except:
                        phone = '전화번호 미등록' # 전화번호가 없을 경우
                    try:
                        homepage = driver.find_element_by_css_selector('a._1RUzg').text # 캠핑장 홈페이지 소스코드
                    except:
                        homepage = '홈페이지 없음' # 캠핑장 홈페이지가 없을 경우



                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser') # 파싱

                    pag = soup.select('#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-of-type(1)')
                    # 리뷰 페이지 소스코드
                    if len(str(pag)) < 20 : # 리뷰 페이지가 없을 경우에는 텍스트를 가져와도 20자가 넘지 않기에 이렇게 설정
                        page = 'not'
                    else:
                        page = str(pag).split('/')[2] # 리뷰 페이지 고유 번호를 split 함수로 가져온다
                    loca_list.append([name,address,phone,homepage,page]) # 이름, 주소, 전화번호, 홈페이지, 리뷰페이지 주소를 리스트에 추가
                    driver.switch_to.default_content() # 다시 캠핑장 정보를 얻기 위해 현재 프레임 나오기
                    driver.switch_to.frame(frame) # 기본 프레임으로 전환
                    time.sleep(1)

                except:
                    pass
            
            if x == 6: # 5번째 페이지부터 다음 페이지 소스 코드가 좀 변하므로 if 설정
                driver.find_element_by_css_selector(f'#app-root > div > div._2lx2y > div._2ky45 > a:nth-child(5)').click()
                time.sleep(7)
                x += 1
            else: # 다음 페이지로 넘어가기 위한 소스코드
                driver.find_element_by_css_selector(f'#app-root > div > div._2lx2y > div._2ky45 > a:nth-child({x})').click()
                time.sleep(7)
                x += 1
            
            driver.switch_to.default_content() # 다음 페이지로 넘어갔을 경우 프레임 전환
            driver.switch_to.frame(frame)
            
            scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[3]/div[1]")
            try:
                for i in range(10):
                    driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div) # 다음 페이지로 넘어간 이후 스크롤 내리기
                    time.sleep(2)
            except:
                pass
    except:
        pass


    columns = ['이름', '주소', '전화' , '홈페이지' , '페이지']
    loca_data = pd.DataFrame(loca_list, columns = columns) # 캠핑장 정보를 모은 리스트로 데이터프레임 생성
    
    return loca_data

In [69]:
def camp_review_data(loca_data): # 캠핑장 리뷰 데이터를 뽑아내기 위한 함수
    loca_review = []
    for i in range(len(loca_data)): # 캠핑장 숫자 만큼 반복
        guitar = loca_data['이름'][i] # 어떤 캠핑장의 리뷰인가를 판명하기 위한 변수
        try : 
            driver.get(f'https://pcmap.place.naver.com/accommodation/{loca_data["페이지"][i]}/review/') # 리뷰 페이지 주소
            time.sleep(7)
            try:
                for a in range(3): # 리뷰 페이지 스크롤 과정
                    time.sleep(1) 
                    driver.find_element_by_tag_name('body').send_keys(Keys.END) 
                    time.sleep(3) 
                    driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.cXO6M > div._2kAri > a').click() 
                    time.sleep(3)
            except:
                pass

            for b in range(50):
                try:
                    driver.find_element_by_css_selector(f' li:nth-child({b+1}) > div._3vfQ6 > a').send_keys(Keys.ENTER)
                    # 리뷰 내용을 완전히 보기 위한 더보기 소스코드
                except:
                    pass

            time.sleep(1)
            reviews = driver.find_element_by_css_selector('div.place_section.cXO6M > div > div > ul') # 리뷰 전체 소스코드

            try:
                for c in range(50):
                    try:
                        review = reviews.find_element_by_css_selector(f'li:nth-child({c+1}) > div._3vfQ6 > a > span').text # 리뷰 소스 코드
                        loca_review.append([review, guitar])
                    except:
                        review = '내용없음' # 별점만 있는등 리뷰 내용이 없을 경우
                        loca_review.append([review, guitar])
            except:
                pass


        except: # 아예 리뷰 페이지 자체가 없을 경우
            review = '내용없음'
            loca_review.append([review, guitar])




    columns = ['리뷰', '해당 캠핑장']
    loca_re_data = pd.DataFrame(loca_review, columns = columns) # 리뷰 데이터 프레임
    return loca_re_data

In [10]:
def tour_data(location, local_data): # 캠핑장 주변 관광지를 찾기 위한 함수
    
    for r in range(len(local_data)): # 캠핑장 숫자 만큼 반복
    
        try:
            driver.get("https://map.naver.com/v5/")
            time.sleep(3)

            try:
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
               ) 
            finally:
                pass

            search_box = driver.find_element_by_class_name("input_search")
            search_box.send_keys(f"{location} {local_data['이름'][r]} 주변 관광지")
            # 같은 캠핑장 이름으로 걸리기도 하므로 location으로 지역을 입력하고 캠핑장 이름에 주변 관광지를 검색어로 입력
            search_box.send_keys(Keys.ENTER) 

            time.sleep(7)

            frame = driver.find_element_by_css_selector("iframe#searchIframe")

            driver.switch_to.frame(frame)
            time.sleep(3)

            loca_list = []

            tour = loca_data['이름'][i]
    
    

            try:
                for i in range(5): # 5개 정도의 주변 관광지 정보를 모음
                    name = driver.find_element_by_xpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i+1}]/div[2]/a[1]/div/div/span[1]').text
                    driver.find_element_by_xpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i+1}]/div[2]/a[1]/div/div/span[1]').click()
                    driver.switch_to.default_content()

                    frame_in = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-place-bridge/div/nm-external-frame-bridge/nm-iframe/iframe')
                    driver.switch_to.frame(frame_in)
                    address = driver.find_element_by_css_selector('span._2yqUQ').text
                    try:
                        phone = driver.find_element_by_css_selector('span._3ZA0S').text
                    except:
                        phone = '전화번호 미등록'
                    try:
                        homepage = driver.find_element_by_css_selector('a._1RUzg').text
                    except:
                        homepage = '홈페이지 없음'



                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    pag = soup.select('#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-of-type(1)')
                    if len(str(pag)) < 20 :
                        page = 'not'
                    else:
                        page = str(pag).split('/')[2]
                    loca_list.append([name,address,phone,homepage,tour,page])
                    driver.switch_to.default_content()
                    driver.switch_to.frame(frame)
                    time.sleep(2)

            except:
                pass
            
   
        except:
            pass


    columns = ['이름', '주소', '전화' , '홈페이지' , '주변 캠핑장', '페이지']
    loca_data = pd.DataFrame(loca_list, columns = columns)
    
    return loca_data

In [11]:
def tour_review_data(loca_data): # 관광지 리뷰 데이터 추출 함수
    loca_review = []
    for i in range(len(loca_data)):
        guitar = loca_data['이름'][i] # 해당 관광지
        tour = loca_data['주변 캠핑장'][i] # 해당 관광지가 어느 캠핑장 주변인가
        try : 
            driver.get(f'https://pcmap.place.naver.com/place/{loca_data["페이지"][i]}/review/')
            time.sleep(7)
            try:
                for a in range(3):
                    time.sleep(1) 
                    driver.find_element_by_tag_name('body').send_keys(Keys.END) 
                    time.sleep(3) 
                    driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.cXO6M > div._2kAri > a').click() 
                    time.sleep(3)
            except:
                pass

            for b in range(50):
                try:
                    driver.find_element_by_css_selector(f' li:nth-child({b+1}) > div._3vfQ6 > a').send_keys(Keys.ENTER)
                except:
                    pass

            time.sleep(1)
            reviews = driver.find_element_by_css_selector('div.place_section.cXO6M > div > div > ul')      

            try:
                for c in range(50):
                    try:
                        review = reviews.find_element_by_css_selector(f'li:nth-child({c+1}) > div._3vfQ6 > a > span').text
                        loca_review.append([review, guitar, tour])
                    except:
                        review = '내용없음'
                        loca_review.append([review, guitar, tour])
            except:
                pass


        except:
            review = '내용없음'
            loca_review.append([review, guitar, tour])




    columns = ['리뷰', '해당 관광지', '주변 캠핑장']
    loca_re_data = pd.DataFrame(loca_review, columns = columns)
    return loca_re_data

In [85]:
# 크롬 드라이버를 켜주는 옵션
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path='./chromedriver',options=chrome_options)

In [ ]:
jeju_data = camp_data('제주', 2)

In [ ]:
jeju_review = camp_review_data(jeju_data)

In [ ]:
gangwon_data = camp_data('강원', 6)

In [ ]:
jeonnam_data = camp_data('전남', 5)

In [ ]:
jeonbuk_data = camp_data('전북', 5)

In [ ]:
gyungnam_data = camp_data('경남', 5)

In [ ]:
gyungbuk_data = camp_data('경북', 5)

In [113]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path='./chromedriver',options=chrome_options)

In [112]:
driver.close()

In [74]:
def searching(local_tour): # 관광지 중에서 캠핑장이 있는지 확인하기 위한 함수
    sp_list = []
    for i in range(len(local_tour)):
        if local_tour["페이지"][i] == 'not': # 애초에 해당 관광지의 페이지가 없을 경우
            pass
        else:
            try:
                driver.get(f'https://pcmap.place.naver.com/place/{local_tour["페이지"][i]}/home?type=list/')
                time.sleep(0.5)
                if '캠핑,야영장' == str(driver.find_element_by_css_selector('span._3ocDE').text): # 소스 코드를 이용해 해당 장소가 캠핑장이라면 빈 리스트에 추가
                    sp_list.append(local_tour['이름'][i])
            except:
                pass
    return sp_list # 캠핑장으로 판명된 주변 관광지의 리스트를 리턴

In [75]:
def searching2(local_tour): # 캠핑장 중에서 비캠핑장이 있는지 확인하기 위한 함수
    sp_list = []
    for i in range(len(local_tour)):
        if local_tour["페이지"][i] == 'not':
            pass
        else:
            try:
                driver.get(f'https://pcmap.place.naver.com/accommodation/{local_tour["페이지"][i]}/home?type=list/')
                time.sleep(0.5)
                if '캠핑,야영장' != str(driver.find_element_by_css_selector('span._3ocDE').text): # 소스 코드를 이용해 해당 장소가 캠핑장이 아니라면 빈 리스트에 추가
                    sp_list.append(local_tour['이름'][i])
            except:
                pass
    return sp_list # 캠핑장이 아닌 장소들의 리스트를 리턴

In [281]:
seoul_camp = pd.read_csv('seoul_camp.csv', index_col=0)
seoul_review = pd.read_csv('seoul_review.csv', index_col=0)
seoul_tour = pd.read_csv('seoul_tour.csv', index_col=0)
seoul_rtour = pd.read_csv('seoul_rtour.csv', index_col=0)
busan_camp = pd.read_csv('busan_camp.csv', index_col=0)
busan_review = pd.read_csv('busan_review.csv', index_col=0)
busan_tour = pd.read_csv('busan_tour.csv', index_col=0)
busan_rtour = pd.read_csv('busan_rtour.csv', index_col=0)
chungbuk_camp = pd.read_csv('chungbuk_camp.csv', index_col=0)
chungbuk_review = pd.read_csv('chungbuk_review.csv', index_col=0)
chungbuk_tour = pd.read_csv('chungbuk_tour.csv', index_col=0)
chungbuk_rtour = pd.read_csv('chungbuk_rtour.csv', index_col=0)
chungnam_camp = pd.read_csv('chungnam_camp.csv', index_col=0)
chungnam_review = pd.read_csv('chungnam_review.csv', index_col=0)
chungnam_tour = pd.read_csv('chungnam_tour.csv', index_col=0)
chungnam_rtour = pd.read_csv('chungnam_rtour.csv', index_col=0)
daegu_camp = pd.read_csv('daegu_camp.csv', index_col=0)
daegu_review = pd.read_csv('daegu_review.csv', index_col=0)
daegu_tour = pd.read_csv('daegu_tour.csv', index_col=0)
daegu_rtour = pd.read_csv('daegu_rtour.csv', index_col=0)
daejun_camp = pd.read_csv('daejun_camp.csv', index_col=0)
daejun_review = pd.read_csv('daejun_review.csv', index_col=0)
daejun_tour = pd.read_csv('daejun_tour.csv', index_col=0)
daejun_rtour = pd.read_csv('daejun_rtour.csv', index_col=0)
gangwon_camp = pd.read_csv('gangwon_camp.csv', index_col=0)
gangwon_review = pd.read_csv('gangwon_review.csv', index_col=0)
gangwon_tour = pd.read_csv('gangwon_tour.csv', index_col=0)
gangwon_rtour = pd.read_csv('gangwon_rtour.csv', index_col=0)
gwangju_camp = pd.read_csv('gwangju_camp.csv', index_col=0)
gwangju_review = pd.read_csv('gwangju_review.csv', index_col=0)
gwangju_tour = pd.read_csv('gwangju_tour.csv', index_col=0)
gwangju_rtour = pd.read_csv('gwangju_rtour.csv', index_col=0)
gyungbuk_camp = pd.read_csv('gyungbuk_camp.csv', index_col=0)
gyungbuk_review = pd.read_csv('gyungbuk_review.csv', index_col=0)
gyungbuk_tour = pd.read_csv('gyungbuk_tour.csv', index_col=0)
gyungbuk_rtour = pd.read_csv('gyungbuk_rtour.csv', index_col=0)
gyungnam_camp = pd.read_csv('gyungnam_camp.csv', index_col=0)
gyungnam_review = pd.read_csv('gyungnam_review.csv', index_col=0)
gyungnam_tour = pd.read_csv('gyungnam_tour.csv', index_col=0)
gyungnam_rtour = pd.read_csv('gyungnam_rtour.csv', index_col=0)
incheon_camp = pd.read_csv('incheon_camp.csv', index_col=0)
incheon_review = pd.read_csv('incheon_review.csv', index_col=0)
incheon_tour = pd.read_csv('incheon_tour.csv', index_col=0)
incheon_rtour = pd.read_csv('incheon_rtour.csv', index_col=0)
jeju_camp = pd.read_csv('jeju_camp.csv', index_col=0)
jeju_review = pd.read_csv('jeju_review.csv', index_col=0)
jeju_tour = pd.read_csv('jeju_tour.csv', index_col=0)
jeju_rtour = pd.read_csv('jeju_rtour.csv', index_col=0)
jeonbuk_camp = pd.read_csv('jeonbuk_camp.csv', index_col=0)
jeonbuk_review = pd.read_csv('jeonbuk_review.csv', index_col=0)
jeonbuk_tour = pd.read_csv('jeonbuk_tour.csv', index_col=0)
jeonbuk_rtour = pd.read_csv('jeonbuk_rtour.csv', index_col=0)
jeonnam_camp = pd.read_csv('jeonnam_camp.csv', index_col=0)
jeonnam_review = pd.read_csv('jeonnam_review.csv', index_col=0)
jeonnam_tour = pd.read_csv('jeonnam_tour.csv', index_col=0)
jeonnam_rtour = pd.read_csv('jeonnam_rtour.csv', index_col=0)
ulsan_camp = pd.read_csv('ulsan_camp.csv', index_col=0)
ulsan_review = pd.read_csv('ulsan_review.csv', index_col=0)
ulsan_tour = pd.read_csv('ulsan_tour.csv', index_col=0)
ulsan_rtour = pd.read_csv('ulsan_rtour.csv', index_col=0)
gyunggi_camp = pd.read_csv('gyunggi_camp.csv', index_col=0)
gyunggi_review = pd.read_csv('gyunggi_review.csv', index_col=0)
gyunggi_tour = pd.read_csv('gyunggi_tour.csv', index_col=0)
gyunggi_rtour = pd.read_csv('gyunggi_rtour.csv', index_col=0)

In [76]:
chungnam1 = searching2(chungnam_camp)
chungnam2 = searching(chungnam_tour)
daegu1 = searching2(daegu_camp)
daegu2 = searching(daegu_tour)
daejun1 = searching2(daejun_camp)
daejun2 = searching(daejun_tour)
gangwon1 = searching2(gangwon_camp)
gangwon2 = searching(gangwon_tour)
gwangju1 = searching2(gwangju_camp)
gwangju2 = searching(gwangju_tour)
# 캠핑장이 아닌 캠핑장 목록과 관광지 목록중 캠핑장인 리스트를 작성

C:\Users\KHJ\AppData\Local\Temp\ipykernel_8424\1742848835.py:10: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  if '캠핑,야영장' != str(driver.find_element_by_css_selector('span._3ocDE').text):


In [106]:
# 만들어둔 리스트를 통해 캠핑장이 아닌 장소들과 리뷰를 제거
for a in busan1: # 리스트 반복
    spac = busan_camp[busan_camp['이름'].str.contains(a)].index # 캠핑장이 아닌 장소들의 인덱스 추출
    busan_camp.drop(spac, inplace=True) # 캠핑장이 아닌 장소 제거
    spac2 = busan_review[busan_review['해당 캠핑장'].str.contains(a)].index # 캠핑장이 아닌 장소들의 리뷰 인덱스 추출
    busan_review.drop(spac2, inplace=True) # 해당 리뷰들 제거

busan_camp = busan_camp.reset_index(drop=True) # 순서 정리
busan_review = busan_review.reset_index(drop=True)

# 만들어둔 리스트를 통해 관광지가 아닌 장소들과 리뷰를 제거
for a in busan2:
    spac = busan_tour[busan_tour['주변 캠핑장'].str.contains(a)].index
    busan_tour.drop(spac, inplace=True)
    spac2 = busan_rtour[busan_rtour['주변 캠핑장'].str.contains(a)].index
    busan_rtour.drop(spac2, inplace=True)

busan_tour = busan_tour.reset_index(drop=True)
busan_rtour = busan_rtour.reset_index(drop=True)

In [107]:
for a in chungnam1:
    spac = chungnam_camp[chungnam_camp['이름'].str.contains(a)].index
    chungnam_camp.drop(spac, inplace=True)
    spac2 = chungnam_review[chungnam_review['해당 캠핑장'].str.contains(a)].index
    chungnam_review.drop(spac2, inplace=True)

chungnam_camp = chungnam_camp.reset_index(drop=True)
chungnam_review = chungnam_review.reset_index(drop=True)

for a in chungnam2:
    spac = chungnam_tour[chungnam_tour['주변 캠핑장'].str.contains(a)].index
    chungnam_tour.drop(spac, inplace=True)
    spac2 = chungnam_rtour[chungnam_rtour['주변 캠핑장'].str.contains(a)].index
    chungnam_rtour.drop(spac2, inplace=True)

chungnam_tour = chungnam_tour.reset_index(drop=True)
chungnam_rtour = chungnam_rtour.reset_index(drop=True)

for a in daegu1:
    spac = daegu_camp[daegu_camp['이름'].str.contains(a)].index
    daegu_camp.drop(spac, inplace=True)
    spac2 = daegu_review[daegu_review['해당 캠핑장'].str.contains(a)].index
    daegu_review.drop(spac2, inplace=True)

daegu_camp = daegu_camp.reset_index(drop=True)
daegu_review = daegu_review.reset_index(drop=True)

for a in daegu2:
    spac = daegu_tour[daegu_tour['주변 캠핑장'].str.contains(a)].index
    daegu_tour.drop(spac, inplace=True)
    spac2 = daegu_rtour[daegu_rtour['주변 캠핑장'].str.contains(a)].index
    daegu_rtour.drop(spac2, inplace=True)

daegu_tour = daegu_tour.reset_index(drop=True)
daegu_rtour = daegu_rtour.reset_index(drop=True)


for a in daejun1:
    spac = daejun_camp[daejun_camp['이름'].str.contains(a)].index
    daejun_camp.drop(spac, inplace=True)
    spac2 = daejun_review[daejun_review['해당 캠핑장'].str.contains(a)].index
    daejun_review.drop(spac2, inplace=True)

daejun_camp = daejun_camp.reset_index(drop=True)
daejun_review = daejun_review.reset_index(drop=True)

for a in daejun2:
    spac = daejun_tour[daejun_tour['주변 캠핑장'].str.contains(a)].index
    daejun_tour.drop(spac, inplace=True)
    spac2 = daejun_rtour[daejun_rtour['주변 캠핑장'].str.contains(a)].index
    daejun_rtour.drop(spac2, inplace=True)

daejun_tour = daejun_tour.reset_index(drop=True)
daejun_rtour = daejun_rtour.reset_index(drop=True)

for a in gangwon1:
    spac = gangwon_camp[gangwon_camp['이름'].str.contains(a)].index
    gangwon_camp.drop(spac, inplace=True)
    spac2 = gangwon_review[gangwon_review['해당 캠핑장'].str.contains(a)].index
    gangwon_review.drop(spac2, inplace=True)

gangwon_camp = gangwon_camp.reset_index(drop=True)
gangwon_review = gangwon_review.reset_index(drop=True)

for a in gangwon2:
    spac = gangwon_tour[gangwon_tour['주변 캠핑장'].str.contains(a)].index
    gangwon_tour.drop(spac, inplace=True)
    spac2 = gangwon_rtour[gangwon_rtour['주변 캠핑장'].str.contains(a)].index
    gangwon_rtour.drop(spac2, inplace=True)

gangwon_tour = gangwon_tour.reset_index(drop=True)
gangwon_rtour = gangwon_rtour.reset_index(drop=True)

for a in gwangju1:
    spac = gwangju_camp[gwangju_camp['이름'].str.contains(a)].index
    gwangju_camp.drop(spac, inplace=True)
    spac2 = gwangju_review[gwangju_review['해당 캠핑장'].str.contains(a)].index
    gwangju_review.drop(spac2, inplace=True)

gwangju_camp = gwangju_camp.reset_index(drop=True)
gwangju_review = gwangju_review.reset_index(drop=True)

for a in gwangju2:
    spac = gwangju_tour[gwangju_tour['주변 캠핑장'].str.contains(a)].index
    gwangju_tour.drop(spac, inplace=True)
    spac2 = gwangju_rtour[gwangju_rtour['주변 캠핑장'].str.contains(a)].index
    gwangju_rtour.drop(spac2, inplace=True)

gwangju_tour = gwangju_tour.reset_index(drop=True)
gwangju_rtour = gwangju_rtour.reset_index(drop=True)


for a in gyunggi1:
    spac = gyunggi_camp[gyunggi_camp['이름'].str.contains(a)].index
    gyunggi_camp.drop(spac, inplace=True)
    spac2 = gyunggi_review[gyunggi_review['해당 캠핑장'].str.contains(a)].index
    gyunggi_review.drop(spac2, inplace=True)

gyunggi_camp = gyunggi_camp.reset_index(drop=True)
gyunggi_review = gyunggi_review.reset_index(drop=True)

for a in gyunggi2:
    spac = gyunggi_tour[gyunggi_tour['주변 캠핑장'].str.contains(a)].index
    gyunggi_tour.drop(spac, inplace=True)
    spac2 = gyunggi_rtour[gyunggi_rtour['주변 캠핑장'].str.contains(a)].index
    gyunggi_rtour.drop(spac2, inplace=True)

gyunggi_tour = gyunggi_tour.reset_index(drop=True)
gyunggi_rtour = gyunggi_rtour.reset_index(drop=True)


for a in gyungbuk1:
    spac = gyungbuk_camp[gyungbuk_camp['이름'].str.contains(a)].index
    gyungbuk_camp.drop(spac, inplace=True)
    spac2 = gyungbuk_review[gyungbuk_review['해당 캠핑장'].str.contains(a)].index
    gyungbuk_review.drop(spac2, inplace=True)

gyungbuk_camp = gyungbuk_camp.reset_index(drop=True)
gyungbuk_review = gyungbuk_review.reset_index(drop=True)

for a in gyungbuk2:
    spac = gyungbuk_tour[gyungbuk_tour['주변 캠핑장'].str.contains(a)].index
    gyungbuk_tour.drop(spac, inplace=True)
    spac2 = gyungbuk_rtour[gyungbuk_rtour['주변 캠핑장'].str.contains(a)].index
    gyungbuk_rtour.drop(spac2, inplace=True)

gyungbuk_tour = gyungbuk_tour.reset_index(drop=True)
gyungbuk_rtour = gyungbuk_rtour.reset_index(drop=True)


for a in gyungnam1:
    spac = gyungnam_camp[gyungnam_camp['이름'].str.contains(a)].index
    gyungnam_camp.drop(spac, inplace=True)
    spac2 = gyungnam_review[gyungnam_review['해당 캠핑장'].str.contains(a)].index
    gyungnam_review.drop(spac2, inplace=True)

gyungnam_camp = gyungnam_camp.reset_index(drop=True)
gyungnam_review = gyungnam_review.reset_index(drop=True)


for a in gyungnam2:
    spac = gyungnam_tour[gyungnam_tour['주변 캠핑장'].str.contains(a)].index
    gyungnam_tour.drop(spac, inplace=True)
    spac2 = gyungnam_rtour[gyungnam_rtour['주변 캠핑장'].str.contains(a)].index
    gyungnam_rtour.drop(spac2, inplace=True)

gyungnam_tour = gyungnam_tour.reset_index(drop=True)
gyungnam_rtour = gyungnam_rtour.reset_index(drop=True)


for a in incheon1:
    spac = incheon_camp[incheon_camp['이름'].str.contains(a)].index
    incheon_camp.drop(spac, inplace=True)
    spac2 = incheon_review[incheon_review['해당 캠핑장'].str.contains(a)].index
    incheon_review.drop(spac2, inplace=True)

incheon_camp = incheon_camp.reset_index(drop=True)
incheon_review = incheon_review.reset_index(drop=True)

for a in incheon2:
    spac = incheon_tour[incheon_tour['주변 캠핑장'].str.contains(a)].index
    incheon_tour.drop(spac, inplace=True)
    spac2 = incheon_rtour[incheon_rtour['주변 캠핑장'].str.contains(a)].index
    incheon_rtour.drop(spac2, inplace=True)

incheon_tour = incheon_tour.reset_index(drop=True)
incheon_rtour = incheon_rtour.reset_index(drop=True)


for a in jeju1:
    spac = jeju_camp[jeju_camp['이름'].str.contains(a)].index
    jeju_camp.drop(spac, inplace=True)
    spac2 = jeju_review[jeju_review['해당 캠핑장'].str.contains(a)].index
    jeju_review.drop(spac2, inplace=True)

jeju_camp = jeju_camp.reset_index(drop=True)
jeju_review = jeju_review.reset_index(drop=True)


for a in jeju2:
    spac = jeju_tour[jeju_tour['주변 캠핑장'].str.contains(a)].index
    jeju_tour.drop(spac, inplace=True)
    spac2 = jeju_rtour[jeju_rtour['주변 캠핑장'].str.contains(a)].index
    jeju_rtour.drop(spac2, inplace=True)

jeju_tour = jeju_tour.reset_index(drop=True)
jeju_rtour = jeju_rtour.reset_index(drop=True)


for a in jeonbuk1:
    spac = jeonbuk_camp[jeonbuk_camp['이름'].str.contains(a)].index
    jeonbuk_camp.drop(spac, inplace=True)
    spac2 = jeonbuk_review[jeonbuk_review['해당 캠핑장'].str.contains(a)].index
    jeonbuk_review.drop(spac2, inplace=True)

jeonbuk_camp = jeonbuk_camp.reset_index(drop=True)
jeonbuk_review = jeonbuk_review.reset_index(drop=True)

for a in jeonbuk2:
    spac = jeonbuk_tour[jeonbuk_tour['주변 캠핑장'].str.contains(a)].index
    jeonbuk_tour.drop(spac, inplace=True)
    spac2 = jeonbuk_rtour[jeonbuk_rtour['주변 캠핑장'].str.contains(a)].index
    jeonbuk_rtour.drop(spac2, inplace=True)

jeonbuk_tour = jeonbuk_tour.reset_index(drop=True)
jeonbuk_rtour = jeonbuk_rtour.reset_index(drop=True)


for a in jeonnam1:
    spac = jeonnam_camp[jeonnam_camp['이름'].str.contains(a)].index
    jeonnam_camp.drop(spac, inplace=True)
    spac2 = jeonnam_review[jeonnam_review['해당 캠핑장'].str.contains(a)].index
    jeonnam_review.drop(spac2, inplace=True)

jeonnam_camp = jeonnam_camp.reset_index(drop=True)
jeonnam_review = jeonnam_review.reset_index(drop=True)

for a in jeonnam2:
    spac = jeonnam_tour[jeonnam_tour['주변 캠핑장'].str.contains(a)].index
    jeonnam_tour.drop(spac, inplace=True)
    spac2 = jeonnam_rtour[jeonnam_rtour['주변 캠핑장'].str.contains(a)].index
    jeonnam_rtour.drop(spac2, inplace=True)

jeonnam_tour = jeonnam_tour.reset_index(drop=True)
jeonnam_rtour = jeonnam_rtour.reset_index(drop=True)


for a in ulsan1:
    spac = ulsan_camp[ulsan_camp['이름'].str.contains(a)].index
    ulsan_camp.drop(spac, inplace=True)
    spac2 = ulsan_review[ulsan_review['해당 캠핑장'].str.contains(a)].index
    ulsan_review.drop(spac2, inplace=True)

ulsan_camp = ulsan_camp.reset_index(drop=True)
ulsan_review = ulsan_review.reset_index(drop=True)


for a in ulsan2:
    spac = ulsan_tour[ulsan_tour['주변 캠핑장'].str.contains(a)].index
    ulsan_tour.drop(spac, inplace=True)
    spac2 = ulsan_rtour[ulsan_rtour['주변 캠핑장'].str.contains(a)].index
    ulsan_rtour.drop(spac2, inplace=True)

ulsan_tour = ulsan_tour.reset_index(drop=True)
ulsan_rtour = ulsan_rtour.reset_index(drop=True)

In [288]:
seoul_camp.to_csv('seoul_camp.csv')
seoul_review.to_csv('seoul_review.csv')
seoul_tour.to_csv('seoul_tour.csv')
seoul_rtour.to_csv('seoul_rtour.csv')

busan_camp.to_csv('busan_camp.csv')
busan_review.to_csv('busan_review.csv')
busan_tour.to_csv('busan_tour.csv')
busan_rtour.to_csv('busan_rtour.csv')

chungbuk_camp.to_csv('chungbuk_camp.csv')
chungbuk_review.to_csv('chungbuk_review.csv')
chungbuk_tour.to_csv('chungbuk_tour.csv')
chungbuk_rtour.to_csv('chungbuk_rtour.csv')

chungnam_camp.to_csv('chungnam_camp.csv')
chungnam_review.to_csv('chungnam_review.csv')
chungnam_tour.to_csv('chungnam_tour.csv')
chungnam_rtour.to_csv('chungnam_rtour.csv')

daegu_camp.to_csv('daegu_camp.csv')
daegu_review.to_csv('daegu_review.csv')
daegu_tour.to_csv('daegu_tour.csv')
daegu_rtour.to_csv('daegu_rtour.csv')

daejun_camp.to_csv('daejun_camp.csv')
daejun_review.to_csv('daejun_review.csv')
daejun_tour.to_csv('daejun_tour.csv')
daejun_rtour.to_csv('daejun_rtour.csv')

gangwon_camp.to_csv('gangwon_camp.csv')
gangwon_review.to_csv('gangwon_review.csv')
gangwon_tour.to_csv('gangwon_tour.csv')
gangwon_rtour.to_csv('gangwon_rtour.csv')

gwangju_camp.to_csv('gwangju_camp.csv')
gwangju_review.to_csv('gwangju_review.csv')
gwangju_tour.to_csv('gwangju_tour.csv')
gwangju_rtour.to_csv('gwangju_rtour.csv')

gyungbuk_camp.to_csv('gyungbuk_camp.csv')
gyungbuk_review.to_csv('gyungbuk_review.csv')
gyungbuk_tour.to_csv('gyungbuk_tour.csv')
gyungbuk_rtour.to_csv('gyungbuk_rtour.csv')

gyunggi_camp.to_csv('gyunggi_camp.csv')
gyunggi_review.to_csv('gyunggi_review.csv')
gyunggi_tour.to_csv('gyunggi_tour.csv')
gyunggi_rtour.to_csv('gyunggi_rtour.csv')

gyungnam_camp.to_csv('gyungnam_camp.csv')
gyungnam_review.to_csv('gyungnam_review.csv')
gyungnam_tour.to_csv('gyungnam_tour.csv')
gyungnam_rtour.to_csv('gyungnam_rtour.csv')

incheon_camp.to_csv('incheon_camp.csv')
incheon_review.to_csv('incheon_review.csv')
incheon_tour.to_csv('incheon_tour.csv')
incheon_rtour.to_csv('incheon_rtour.csv')

jeju_camp.to_csv('jeju_camp.csv')
jeju_review.to_csv('jeju_review.csv')
jeju_tour.to_csv('jeju_tour.csv')
jeju_rtour.to_csv('jeju_rtour.csv')

jeonbuk_camp.to_csv('jeonbuk_camp.csv')
jeonbuk_review.to_csv('jeonbuk_review.csv')
jeonbuk_tour.to_csv('jeonbuk_tour.csv')
jeonbuk_rtour.to_csv('jeonbuk_rtour.csv')

jeonnam_camp.to_csv('jeonnam_camp.csv')
jeonnam_review.to_csv('jeonnam_review.csv')
jeonnam_tour.to_csv('jeonnam_tour.csv')
jeonnam_rtour.to_csv('jeonnam_rtour.csv')

ulsan_camp.to_csv('ulsan_camp.csv')
ulsan_review.to_csv('ulsan_review.csv')
ulsan_tour.to_csv('ulsan_tour.csv')
ulsan_rtour.to_csv('ulsan_rtour.csv')


In [80]:
def jeonchuri(local_review, local_rtour): # 리뷰 내용들을 적절히 전처리하는 함수
    local_review = local_review.dropna() # 결측치 처리
    local_rtour = local_rtour.dropna()
    local_rtour = local_rtour.reset_index(drop=True)
    local_review = local_review.reset_index(drop=True)

    for i in range(len(local_review)):
        local_review['리뷰'][i] = local_review['리뷰'][i].replace('\n', ' ') # 줄이 바뀔때의 \n을 제거

    for i in range(len(local_rtour)):
        local_rtour['리뷰'][i] = local_rtour['리뷰'][i].replace('\n', ' ')
    
    local_review, local_rtour


def jeonchuri2(local_camp, local_review, local_tour, local_rtour): # 전처리 함수
    local_camp = local_camp.drop(['페이지'], axis=1) # 리뷰들을 수집하기 위해 만들어뒀던 페이지 컬럼을 제거

    local_review = local_review[local_review['리뷰'] != '[]']
    local_review = local_review[local_review['리뷰'] != '내용없음']
    local_review = local_review[local_review['리뷰'] != 'ㆍ']
    local_rtour = local_rtour[local_rtour['리뷰'] != 'ㆍ'] # 필요없는 리뷰들 제거


    local_review = local_review.drop_duplicates() # 중복 제거
    local_review = local_review.reset_index(drop=True)

    local_tour = local_tour.drop(['페이지'], axis=1)

    local_rtour = local_rtour[local_rtour['리뷰'] != '[]']
    local_rtour = local_rtour[local_rtour['리뷰'] != '내용없음']
    local_rtour = local_rtour.drop_duplicates()
    local_rtour = local_rtour.reset_index(drop=True)

    return local_camp, local_review, local_tour, local_rtour

In [283]:
korea_camp = pd.concat([seoul_camp, busan_camp, chungbuk_camp, chungnam_camp, daegu_camp, daejun_camp, gangwon_camp, gwangju_camp, gyunggi_camp, gyungbuk_camp, gyungnam_camp, incheon_camp, jeju_camp, ulsan_camp, jeonbuk_camp, jeonnam_camp])
korea_camp = korea_camp.reset_index(drop=True)
korea_camp
# 캠핑장 데이터셋 합치기

,이름,주소,전화,홈페이지
0,서울북한산럭셔리카라반&수영장글램핑,서울 은평구 북한산로 242,070-4336-1824,https://bookhansan.campingt.me/
1,난지캠핑장,서울 마포구 한강난지로 28,02-373-2021,https://yeyak.seoul.go.kr
2,노을캠핑장,서울 마포구 하늘공원로 108-1,02-304-3213,http://parks.seoul.go.kr/template/sub/worldcup...
3,중랑캠핑숲,서울 중랑구 망우로87길 110,0507-1348-8138,http://www.joongrangsoop.com
4,강동그린웨이 가족캠핑장,서울 강동구 천호대로206길 87,02-478-4079,https://www.igangdong.or.kr/page/ce/ce_0201.asp
...,...,...,...,...
1359,섬진강문화학교,전남 곡성군 죽곡면 태안로 793,전화번호 미등록,홈페이지 없음
1360,담양대나무골야영장,전남 담양군 금성면 비내동길 148,061-383-9291,홈페이지 없음
1361,땅끝마을글램핑,전남 해남군 송지면 갈산길 95,010-2433-4688,http://onetwogogo.co.kr/
1362,비금도원평해변캠핑장,전남 신안군 비금면 신원리,061-271-8605,홈페이지 없음


In [284]:
korea_review = pd.concat([seoul_review, busan_review, chungbuk_review, chungnam_review, daegu_review, daejun_review, gangwon_review, gwangju_review, gyunggi_review, gyungbuk_review, gyungnam_review, incheon_review, jeju_review, ulsan_review, jeonbuk_review, jeonnam_review])
korea_review = korea_review.reset_index(drop=True)
korea_review
# 캠핑장 리뷰 데이터셋 합치기

,리뷰,해당 캠핑장
0,어린이날이라 어딜가나 고민하다 미리예약했습니다. 카라반내부와 침구가 너무깨끗해서 만...,서울북한산럭셔리카라반&수영장글램핑
1,좋았습니다~! 깔끔했구요 조용했어요~! 주차장부터 카라반앞까지 픽업해주셔서 감사했습...,서울북한산럭셔리카라반&수영장글램핑
2,다른 글램핑 여럿 가봤는데 여기가 제일 좋았어요! 1. 매우 깔끔/친절 : 연식이 ...,서울북한산럭셔리카라반&수영장글램핑
3,진심으로 너무너무너무 예쁘고 깨끗하고 좋은곳이였어요!!!!! 벌레가 많은건 당연한거...,서울북한산럭셔리카라반&수영장글램핑
4,주차장에 도착하자마자 카트로 픽업해주시고 잘 안내해주셨어요. 내부공간은 넷이서 쓰기...,서울북한산럭셔리카라반&수영장글램핑
...,...,...
26799,오 좋아요,낙안민속자연휴양림야영장
26800,또 쉬러가고 싶은곳,방장산자연휴양림캠핑장
26801,시설 깨끗하고 이용하기 아주 편리하고 좋아요,방장산자연휴양림캠핑장
26802,좋아요,방장산자연휴양림캠핑장


In [285]:
korea_tour = pd.concat([seoul_tour, busan_tour, chungbuk_tour, chungnam_tour, daegu_tour, daejun_tour, gangwon_tour, gwangju_tour, gyunggi_tour, gyungbuk_tour, gyungnam_tour, incheon_tour, jeju_tour, ulsan_tour, jeonbuk_tour, jeonnam_tour])
korea_tour = korea_tour.reset_index(drop=True)
korea_tour
# 주변 관광지 데이터셋 합치기

,이름,주소,전화,홈페이지,주변 캠핑장
0,대서문,서울 은평구 진관동 251-2,전화번호 미등록,홈페이지 없음,서울북한산럭셔리카라반&수영장글램핑
1,북한산둘레길10구간,서울 은평구 진관동 268,전화번호 미등록,홈페이지 없음,서울북한산럭셔리카라반&수영장글램핑
2,은평한옥마을,서울 은평구 진관동 193-14,전화번호 미등록,https://eptour.kr,서울북한산럭셔리카라반&수영장글램핑
3,난지 한강공원,서울 마포구 한강난지로 162 한강공원 난지안내센터,02-3780-0611,http://hangang.seoul.go.kr/archives/46777,난지캠핑장
4,하늘공원,서울 마포구 하늘공원로 95,02-300-5501,지식백과 더보기,난지캠핑장
...,...,...,...,...,...
2781,명사십리해변,전남 신안군 비금면 구림리,전화번호 미등록,홈페이지 없음,비금도원평해변캠핑장
2782,원평해변,전남 신안군 비금면 원평해수욕장길 64,061-240-8586,지식백과 더보기,비금도원평해변캠핑장
2783,원평항등대,전남 신안군 비금면 신원리,전화번호 미등록,홈페이지 없음,비금도원평해변캠핑장
2784,지리산구례수목원,전남 구례군 산동면 탑동1길 125,061-783-0599,http://ecopark.gurye.go.kr/,숲속캠핑장


In [286]:
korea_rtour = pd.concat([seoul_rtour, busan_rtour, chungbuk_rtour, chungnam_rtour, daegu_rtour, daejun_rtour, gangwon_rtour, gwangju_rtour, gyunggi_rtour, gyungbuk_rtour, gyungnam_rtour, incheon_rtour, jeju_rtour, ulsan_rtour, jeonbuk_rtour, jeonnam_rtour])
korea_rtour = korea_rtour.reset_index(drop=True)
korea_rtour
# 주변 관광지 리뷰 데이터셋 합치기

,리뷰,해당 관광지,주변 캠핑장
0,노을 캠핑장 다녀왔는데 너무 좋네요~,노을공원,노을캠핑장
1,노을캠핑장 최고입니다,노을공원,노을캠핑장
2,캠핑왔는데 주차료 비싸네요~~~;;;,노을공원,노을캠핑장
3,가까운곳에서 자연을 느낄수있어요,노을공원,노을캠핑장
4,굿~,노을공원,노을캠핑장
...,...,...,...
14381,오랜만에 저혼자 힐링하고 가네요 ㅋㅋ 그날은 1~2명 빼고는 저 혼자 전세낸듯. ㅋ...,석정온천휴스파,방장산자연휴양림캠핑장
14382,남탕은 탕에서 마스크를 아무도 안쓰고 있네요.,석정온천휴스파,방장산자연휴양림캠핑장
14383,가족모두 수영을 좋아해서 가끔 찾는 곳입니다. 온천도 좋지만 오늘은 눈치게임도 완전...,석정온천휴스파,방장산자연휴양림캠핑장
14384,늘 즣은 온천수로 힐링이 되는 장소입니다.,석정온천휴스파,방장산자연휴양림캠핑장


In [287]:
korea_camp.to_csv('korea_camp.csv')
korea_review.to_csv('korea_review.csv')
korea_tour.to_csv('korea_tour.csv')
korea_rtour.to_csv('korea_rtour.csv')

In [17]:
seoul_rev = pd.DataFrame(seoul_review['해당 캠핑장'].value_counts()) # 캠핑장별 리뷰 숫자를 데이터프레임화
seoul_rev['캠핑장'] = seoul_rev.index
seoul_rev = seoul_rev.reset_index(drop=True)
seoul1 = list(seoul_rev['캠핑장'][seoul_rev['해당 캠핑장'] < 10])
# 리뷰 숫자가 적은 캠핑장 리스트 작성

In [ ]:
daegu_rev = pd.DataFrame(daegu_review['해당 캠핑장'].value_counts())
daegu_rev['이름'] = daegu_rev.index
daegu_rev = daegu_rev.reset_index(drop=True)
daegu_camp1 = pd.merge(daegu_camp, daegu_rev, how='outer',on='이름')
daegu_camp1 = daegu_camp1.fillna(0)
daegu1 = list(daegu_camp1['이름'][daegu_camp1['해당 캠핑장'] == 0])
daegu1.remove('팔공산도학캠핑장')


chungbuk_rev = pd.DataFrame(chungbuk_review['해당 캠핑장'].value_counts())
chungbuk_rev['이름'] = chungbuk_rev.index
chungbuk_rev = chungbuk_rev.reset_index(drop=True)
chungbuk_camp1 = pd.merge(chungbuk_camp, chungbuk_rev, how='outer',on='이름')
chungbuk_camp1 = chungbuk_camp1.fillna(0)
chungbuk1 = list(chungbuk_camp1['이름'][chungbuk_camp1['해당 캠핑장'] < 40]) # 리뷰 숫자가 40개 아닌 캠핑장 리스트
chungbuk2 = list(chungbuk_camp1['이름'][chungbuk_camp1['해당 캠핑장'] == 40])
del chungbuk2[:35] # 35개의 캠핑장 데이터를 남기기 위한 코드
chungbuk1 = chungbuk1 + chungbuk2
len(chungbuk1)

chungnam_rev = pd.DataFrame(chungnam_review['해당 캠핑장'].value_counts())
chungnam_rev['이름'] = chungnam_rev.index
chungnam_rev = chungnam_rev.reset_index(drop=True)
chungnam_camp1 = pd.merge(chungnam_camp, chungnam_rev, how='outer',on='이름')
chungnam_camp1 = chungnam_camp1.fillna(0)
chungnam1 = list(chungnam_camp1['이름'][chungnam_camp1['해당 캠핑장'] < 40])
chungnam2 = list(chungnam_camp1['이름'][chungnam_camp1['해당 캠핑장'] == 40])
del chungnam2[:30]
chungnam1 = chungnam1 + chungnam2
len(chungnam1)

gangwon_rev = pd.DataFrame(gangwon_review['해당 캠핑장'].value_counts())
gangwon_rev['이름'] = gangwon_rev.index
gangwon_rev = gangwon_rev.reset_index(drop=True)
gangwon_camp1 = pd.merge(gangwon_camp, gangwon_rev, how='outer',on='이름')
gangwon_camp1 = gangwon_camp1.fillna(0)
gangwon1 = list(gangwon_camp1['이름'][gangwon_camp1['해당 캠핑장'] < 40])
gangwon2 = list(gangwon_camp1['이름'][gangwon_camp1['해당 캠핑장'] == 40])
del gangwon2[:42]
gangwon1 = gangwon1 + gangwon2
len(gangwon1)

gyunggi_rev = pd.DataFrame(gyunggi_review['해당 캠핑장'].value_counts())
gyunggi_rev['이름'] = gyunggi_rev.index
gyunggi_rev = gyunggi_rev.reset_index(drop=True)
gyunggi_camp1 = pd.merge(gyunggi_camp, gyunggi_rev, how='outer',on='이름')
gyunggi_camp1 = gyunggi_camp1.fillna(0)
gyunggi1 = list(gyunggi_camp1['이름'][gyunggi_camp1['해당 캠핑장'] < 40])
gyunggi2 = list(gyunggi_camp1['이름'][gyunggi_camp1['해당 캠핑장'] == 40])
del gyunggi2[:40]
gyunggi1 = gyunggi1 + gyunggi2
len(gyunggi1)

gyungbuk_rev = pd.DataFrame(gyungbuk_review['해당 캠핑장'].value_counts())
gyungbuk_rev['이름'] = gyungbuk_rev.index
gyungbuk_rev = gyungbuk_rev.reset_index(drop=True)
gyungbuk_camp1 = pd.merge(gyungbuk_camp, gyungbuk_rev, how='outer',on='이름')
gyungbuk_camp1 = gyungbuk_camp1.fillna(0)
gyungbuk1 = list(gyungbuk_camp1['이름'][gyungbuk_camp1['해당 캠핑장'] < 40])
gyungbuk2 = list(gyungbuk_camp1['이름'][gyungbuk_camp1['해당 캠핑장'] == 40])
del gyungbuk2[:34]
gyungbuk1 = gyungbuk1 + gyungbuk2
len(gyungbuk1)

gyungnam_rev = pd.DataFrame(gyungnam_review['해당 캠핑장'].value_counts())
gyungnam_rev['이름'] = gyungnam_rev.index
gyungnam_rev = gyungnam_rev.reset_index(drop=True)
gyungnam_camp1 = pd.merge(gyungnam_camp, gyungnam_rev, how='outer',on='이름')
gyungnam_camp1 = gyungnam_camp1.fillna(0)
gyungnam1 = list(gyungnam_camp1['이름'][gyungnam_camp1['해당 캠핑장'] < 40])
gyungnam2 = list(gyungnam_camp1['이름'][gyungnam_camp1['해당 캠핑장'] == 40])
del gyungnam2[:36]
gyungnam1 = gyungnam1 + gyungnam2
len(gyungnam1)

incheon_rev = pd.DataFrame(incheon_review['해당 캠핑장'].value_counts())
incheon_rev['이름'] = incheon_rev.index
incheon_rev = incheon_rev.reset_index(drop=True)
incheon_camp1 = pd.merge(incheon_camp, incheon_rev, how='outer',on='이름')
incheon_camp1 = incheon_camp1.fillna(0)
incheon1 = list(incheon_camp1['이름'][incheon_camp1['해당 캠핑장'] < 5])


jeju_rev = pd.DataFrame(jeju_review['해당 캠핑장'].value_counts())
jeju_rev['이름'] = jeju_rev.index
jeju_rev = jeju_rev.reset_index(drop=True)
jeju_camp1 = pd.merge(jeju_camp, jeju_rev, how='outer',on='이름')
jeju_camp1 = jeju_camp1.fillna(0)
jeju1 = list(jeju_camp1['이름'][jeju_camp1['해당 캠핑장'] < 10])

jeonbuk_rev = pd.DataFrame(jeonbuk_review['해당 캠핑장'].value_counts())
jeonbuk_rev['이름'] = jeonbuk_rev.index
jeonbuk_rev = jeonbuk_rev.reset_index(drop=True)
jeonbuk_camp1 = pd.merge(jeonbuk_camp, jeonbuk_rev, how='outer',on='이름')
jeonbuk_camp1 = jeonbuk_camp1.fillna(0)
jeonbuk1 = list(jeonbuk_camp1['이름'][jeonbuk_camp1['해당 캠핑장'] < 40])

jeonnam_rev = pd.DataFrame(jeonnam_review['해당 캠핑장'].value_counts())
jeonnam_rev['이름'] = jeonnam_rev.index
jeonnam_rev = jeonnam_rev.reset_index(drop=True)
jeonnam_camp1 = pd.merge(jeonnam_camp, jeonnam_rev, how='outer',on='이름')
jeonnam_camp1 = jeonnam_camp1.fillna(0)
jeonnam1 = list(jeonnam_camp1['이름'][jeonnam_camp1['해당 캠핑장'] < 40])


ulsan_rev = pd.DataFrame(ulsan_review['해당 캠핑장'].value_counts())
ulsan_rev['이름'] = ulsan_rev.index
ulsan_rev = ulsan_rev.reset_index(drop=True)
ulsan_camp1 = pd.merge(ulsan_camp, ulsan_rev, how='outer',on='이름')
ulsan_camp1 = ulsan_camp1.fillna(0)
ulsan1 = list(ulsan_camp1['이름'][ulsan_camp1['해당 캠핑장'] < 5])

In [90]:
for a in seoul1: # 리뷰 숫자에 따라서 캠핑장들을 제거
    spac = seoul_camp[seoul_camp['이름'].str.contains(a)].index # 리뷰 숫자가 모자른 캠핑장 인덱스
    seoul_camp.drop(spac, inplace=True) # 해당 캠핑장 제거
    spac2 = seoul_review[seoul_review['해당 캠핑장'].str.contains(a)].index
    seoul_review.drop(spac2, inplace=True)
    spac3 = seoul_tour[seoul_tour['주변 캠핑장'].str.contains(a)].index
    seoul_tour.drop(spac3, inplace=True)
    spac4 = seoul_rtour[seoul_rtour['주변 캠핑장'].str.contains(a)].index
    seoul_rtour.drop(spac4, inplace=True)

seoul_camp = seoul_camp.reset_index(drop=True)
seoul_review = seoul_review.reset_index(drop=True)

seoul_tour = seoul_tour.reset_index(drop=True)
seoul_rtour = seoul_rtour.reset_index(drop=True)

In [182]:
for a in daegu1:
    spac = daegu_camp[daegu_camp['이름'].str.contains(a)].index
    daegu_camp.drop(spac, inplace=True)
    spac2 = daegu_review[daegu_review['해당 캠핑장'].str.contains(a)].index
    daegu_review.drop(spac2, inplace=True)
    spac3 = daegu_tour[daegu_tour['주변 캠핑장'].str.contains(a)].index
    daegu_tour.drop(spac3, inplace=True)
    spac4 = daegu_rtour[daegu_rtour['주변 캠핑장'].str.contains(a)].index
    daegu_rtour.drop(spac4, inplace=True)
daegu_camp = daegu_camp.reset_index(drop=True)
daegu_review = daegu_review.reset_index(drop=True)
daegu_tour = daegu_tour.reset_index(drop=True)
daegu_rtour = daegu_rtour.reset_index(drop=True)


for a in ulsan1:
    spac = ulsan_camp[ulsan_camp['이름'].str.contains(a)].index
    ulsan_camp.drop(spac, inplace=True)
    spac2 = ulsan_review[ulsan_review['해당 캠핑장'].str.contains(a)].index
    ulsan_review.drop(spac2, inplace=True)
    spac3 = ulsan_tour[ulsan_tour['주변 캠핑장'].str.contains(a)].index
    ulsan_tour.drop(spac3, inplace=True)
    spac4 = ulsan_rtour[ulsan_rtour['주변 캠핑장'].str.contains(a)].index
    ulsan_rtour.drop(spac4, inplace=True)
ulsan_camp = ulsan_camp.reset_index(drop=True)
ulsan_review = ulsan_review.reset_index(drop=True)
ulsan_tour = ulsan_tour.reset_index(drop=True)
ulsan_rtour = ulsan_rtour.reset_index(drop=True)

for a in chungbuk1:
    spac = chungbuk_camp[chungbuk_camp['이름'].str.contains(a)].index
    chungbuk_camp.drop(spac, inplace=True)
    spac2 = chungbuk_review[chungbuk_review['해당 캠핑장'].str.contains(a)].index
    chungbuk_review.drop(spac2, inplace=True)
    spac3 = chungbuk_tour[chungbuk_tour['주변 캠핑장'].str.contains(a)].index
    chungbuk_tour.drop(spac3, inplace=True)
    spac4 = chungbuk_rtour[chungbuk_rtour['주변 캠핑장'].str.contains(a)].index
    chungbuk_rtour.drop(spac4, inplace=True)
chungbuk_camp = chungbuk_camp.reset_index(drop=True)
chungbuk_review = chungbuk_review.reset_index(drop=True)
chungbuk_tour = chungbuk_tour.reset_index(drop=True)
chungbuk_rtour = chungbuk_rtour.reset_index(drop=True)


for a in chungnam1:
    spac = chungnam_camp[chungnam_camp['이름'].str.contains(a)].index
    chungnam_camp.drop(spac, inplace=True)
    spac2 = chungnam_review[chungnam_review['해당 캠핑장'].str.contains(a)].index
    chungnam_review.drop(spac2, inplace=True)
    spac3 = chungnam_tour[chungnam_tour['주변 캠핑장'].str.contains(a)].index
    chungnam_tour.drop(spac3, inplace=True)
    spac4 = chungnam_rtour[chungnam_rtour['주변 캠핑장'].str.contains(a)].index
    chungnam_rtour.drop(spac4, inplace=True)
chungnam_camp = chungnam_camp.reset_index(drop=True)
chungnam_review = chungnam_review.reset_index(drop=True)
chungnam_tour = chungnam_tour.reset_index(drop=True)
chungnam_rtour = chungnam_rtour.reset_index(drop=True)


for a in gangwon1:
    spac = gangwon_camp[gangwon_camp['이름'].str.contains(a)].index
    gangwon_camp.drop(spac, inplace=True)
    spac2 = gangwon_review[gangwon_review['해당 캠핑장'].str.contains(a)].index
    gangwon_review.drop(spac2, inplace=True)
    spac3 = gangwon_tour[gangwon_tour['주변 캠핑장'].str.contains(a)].index
    gangwon_tour.drop(spac3, inplace=True)
    spac4 = gangwon_rtour[gangwon_rtour['주변 캠핑장'].str.contains(a)].index
    gangwon_rtour.drop(spac4, inplace=True)
gangwon_camp = gangwon_camp.reset_index(drop=True)
gangwon_review = gangwon_review.reset_index(drop=True)
gangwon_tour = gangwon_tour.reset_index(drop=True)
gangwon_rtour = gangwon_rtour.reset_index(drop=True)



for a in gyungbuk1:
    spac = gyungbuk_camp[gyungbuk_camp['이름'].str.contains(a)].index
    gyungbuk_camp.drop(spac, inplace=True)
    spac2 = gyungbuk_review[gyungbuk_review['해당 캠핑장'].str.contains(a)].index
    gyungbuk_review.drop(spac2, inplace=True)
    spac3 = gyungbuk_tour[gyungbuk_tour['주변 캠핑장'].str.contains(a)].index
    gyungbuk_tour.drop(spac3, inplace=True)
    spac4 = gyungbuk_rtour[gyungbuk_rtour['주변 캠핑장'].str.contains(a)].index
    gyungbuk_rtour.drop(spac4, inplace=True)
gyungbuk_camp = gyungbuk_camp.reset_index(drop=True)
gyungbuk_review = gyungbuk_review.reset_index(drop=True)
gyungbuk_tour = gyungbuk_tour.reset_index(drop=True)
gyungbuk_rtour = gyungbuk_rtour.reset_index(drop=True)


for a in gyungnam1:
    spac = gyungnam_camp[gyungnam_camp['이름'].str.contains(a)].index
    gyungnam_camp.drop(spac, inplace=True)
    spac2 = gyungnam_review[gyungnam_review['해당 캠핑장'].str.contains(a)].index
    gyungnam_review.drop(spac2, inplace=True)
    spac3 = gyungnam_tour[gyungnam_tour['주변 캠핑장'].str.contains(a)].index
    gyungnam_tour.drop(spac3, inplace=True)
    spac4 = gyungnam_rtour[gyungnam_rtour['주변 캠핑장'].str.contains(a)].index
    gyungnam_rtour.drop(spac4, inplace=True)
gyungnam_camp = gyungnam_camp.reset_index(drop=True)
gyungnam_review = gyungnam_review.reset_index(drop=True)
gyungnam_tour = gyungnam_tour.reset_index(drop=True)
gyungnam_rtour = gyungnam_rtour.reset_index(drop=True)


for a in gyunggi1:
    spac = gyunggi_camp[gyunggi_camp['이름'].str.contains(a)].index
    gyunggi_camp.drop(spac, inplace=True)
    spac2 = gyunggi_review[gyunggi_review['해당 캠핑장'].str.contains(a)].index
    gyunggi_review.drop(spac2, inplace=True)
    spac3 = gyunggi_tour[gyunggi_tour['주변 캠핑장'].str.contains(a)].index
    gyunggi_tour.drop(spac3, inplace=True)
    spac4 = gyunggi_rtour[gyunggi_rtour['주변 캠핑장'].str.contains(a)].index
    gyunggi_rtour.drop(spac4, inplace=True)
gyunggi_camp = gyunggi_camp.reset_index(drop=True)
gyunggi_review = gyunggi_review.reset_index(drop=True)
gyunggi_tour = gyunggi_tour.reset_index(drop=True)
gyunggi_rtour = gyunggi_rtour.reset_index(drop=True)



for a in incheon1:
    spac = incheon_camp[incheon_camp['이름'].str.contains(a)].index
    incheon_camp.drop(spac, inplace=True)
    spac2 = incheon_review[incheon_review['해당 캠핑장'].str.contains(a)].index
    incheon_review.drop(spac2, inplace=True)
    spac3 = incheon_tour[incheon_tour['주변 캠핑장'].str.contains(a)].index
    incheon_tour.drop(spac3, inplace=True)
    spac4 = incheon_rtour[incheon_rtour['주변 캠핑장'].str.contains(a)].index
    incheon_rtour.drop(spac4, inplace=True)
incheon_camp = incheon_camp.reset_index(drop=True)
incheon_review = incheon_review.reset_index(drop=True)
incheon_tour = incheon_tour.reset_index(drop=True)
incheon_rtour = incheon_rtour.reset_index(drop=True)


for a in jeju1:
    spac = jeju_camp[jeju_camp['이름'].str.contains(a)].index
    jeju_camp.drop(spac, inplace=True)
    spac2 = jeju_review[jeju_review['해당 캠핑장'].str.contains(a)].index
    jeju_review.drop(spac2, inplace=True)
    spac3 = jeju_tour[jeju_tour['주변 캠핑장'].str.contains(a)].index
    jeju_tour.drop(spac3, inplace=True)
    spac4 = jeju_rtour[jeju_rtour['주변 캠핑장'].str.contains(a)].index
    jeju_rtour.drop(spac4, inplace=True)
jeju_camp = jeju_camp.reset_index(drop=True)
jeju_review = jeju_review.reset_index(drop=True)
jeju_tour = jeju_tour.reset_index(drop=True)
jeju_rtour = jeju_rtour.reset_index(drop=True)



for a in jeonbuk1:
    spac = jeonbuk_camp[jeonbuk_camp['이름'].str.contains(a)].index
    jeonbuk_camp.drop(spac, inplace=True)
    spac2 = jeonbuk_review[jeonbuk_review['해당 캠핑장'].str.contains(a)].index
    jeonbuk_review.drop(spac2, inplace=True)
    spac3 = jeonbuk_tour[jeonbuk_tour['주변 캠핑장'].str.contains(a)].index
    jeonbuk_tour.drop(spac3, inplace=True)
    spac4 = jeonbuk_rtour[jeonbuk_rtour['주변 캠핑장'].str.contains(a)].index
    jeonbuk_rtour.drop(spac4, inplace=True)
jeonbuk_camp = jeonbuk_camp.reset_index(drop=True)
jeonbuk_review = jeonbuk_review.reset_index(drop=True)
jeonbuk_tour = jeonbuk_tour.reset_index(drop=True)
jeonbuk_rtour = jeonbuk_rtour.reset_index(drop=True)


for a in jeonnam1:
    spac = jeonnam_camp[jeonnam_camp['이름'].str.contains(a)].index
    jeonnam_camp.drop(spac, inplace=True)
    spac2 = jeonnam_review[jeonnam_review['해당 캠핑장'].str.contains(a)].index
    jeonnam_review.drop(spac2, inplace=True)
    spac3 = jeonnam_tour[jeonnam_tour['주변 캠핑장'].str.contains(a)].index
    jeonnam_tour.drop(spac3, inplace=True)
    spac4 = jeonnam_rtour[jeonnam_rtour['주변 캠핑장'].str.contains(a)].index
    jeonnam_rtour.drop(spac4, inplace=True)
jeonnam_camp = jeonnam_camp.reset_index(drop=True)
jeonnam_review = jeonnam_review.reset_index(drop=True)
jeonnam_tour = jeonnam_tour.reset_index(drop=True)
jeonnam_rtour = jeonnam_rtour.reset_index(drop=True)







C:\Users\KHJ\AppData\Local\Temp\ipykernel_8412\2091037696.py:61: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  spac = gangwon_camp[gangwon_camp['이름'].str.contains(a)].index
C:\Users\KHJ\AppData\Local\Temp\ipykernel_8412\2091037696.py:63: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  spac2 = gangwon_review[gangwon_review['해당 캠핑장'].str.contains(a)].index
C:\Users\KHJ\AppData\Local\Temp\ipykernel_8412\2091037696.py:65: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  spac3 = gangwon_tour[gangwon_tour['주변 캠핑장'].str.contains(a)].index
C:\Users\KHJ\AppData\Local\Temp\ipykernel_8412\2091037696.py:67: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  

In [257]:
def dele(local_camp, local_review, local_tour, local_rtour): # 캠핑장 항목에 없는 리뷰, 관광지, 관광지 리뷰를 제거하기 위한 전처리 함수
    local1 = list(local_camp['이름'])
    local2 = list(set(local_tour['주변 캠핑장']))
    local3 = list(set(local2) - set(local1)) # 리뷰가 있는 캠핑장과 캠핑장 데이터셋의 차집합
    for a in local3: # 차집합으로 구한 리스트를 반복
        spac = local_tour[local_tour['주변 캠핑장'].str.contains(a)].index # 리뷰에만 있던 캠핑장들의 인덱스
        local_tour.drop(spac, inplace=True) # 해당 캠핑장 제거
    local4 = list(set(local_review['해당 캠핑장']))
    local5 = list(set(local4) - set(local1))
    for a in local5:
        spac2 = local_review[local_review['해당 캠핑장'].str.contains(a)].index
        local_review.drop(spac2, inplace=True)
    local6 = list(set(local_rtour['주변 캠핑장']))
    local7 = list(set(local6) - set(local1))
    for a in local7:
        spac4 = local_rtour[local_rtour['주변 캠핑장'].str.contains(a)].index
        local_rtour.drop(spac4, inplace=True)

    local_review = local_review.reset_index(drop=True)
    local_tour = local_tour.reset_index(drop=True)
    local_rtour = local_rtour.reset_index(drop=True)
    return local_review, local_tour, local_rtour

In [282]:
daegu_review, daegu_tour, daegu_rtour = dele(daegu_camp, daegu_review, daegu_tour, daegu_rtour)
busan_review, busan_tour, busan_rtour = dele(busan_camp, busan_review, busan_tour, busan_rtour)
chungbuk_review, chungbuk_tour, chungbuk_rtour = dele(chungbuk_camp, chungbuk_review, chungbuk_tour, chungbuk_rtour)
chungnam_review, chungnam_tour, chungnam_rtour = dele(chungnam_camp, chungnam_review, chungnam_tour, chungnam_rtour)
daejun_review, daejun_tour, daejun_rtour = dele(daejun_camp, daejun_review, daejun_tour, daejun_rtour)
gwangju_review, gwangju_tour, gwangju_rtour = dele(gwangju_camp, gwangju_review, gwangju_tour, gwangju_rtour)
gangwon_review, gangwon_tour, gangwon_rtour = dele(gangwon_camp, gangwon_review, gangwon_tour, gangwon_rtour)
gyunggi_review, gyunggi_tour, gyunggi_rtour = dele(gyunggi_camp, gyunggi_review, gyunggi_tour, gyunggi_rtour)
gyungbuk_review, gyungbuk_tour, gyungbuk_rtour = dele(gyungbuk_camp, gyungbuk_review, gyungbuk_tour, gyungbuk_rtour)
gyungnam_review, gyungnam_tour, gyungnam_rtour = dele(gyungnam_camp, gyungnam_review, gyungnam_tour, gyungnam_rtour)
incheon_review, incheon_tour, incheon_rtour = dele(incheon_camp, incheon_review, incheon_tour, incheon_rtour)
jeju_review, jeju_tour, jeju_rtour = dele(jeju_camp, jeju_review, jeju_tour, jeju_rtour)
jeonbuk_review, jeonbuk_tour, jeonbuk_rtour = dele(jeonbuk_camp, jeonbuk_review, jeonbuk_tour, jeonbuk_rtour)
jeonnam_review, jeonnam_tour, jeonnam_rtour = dele(jeonnam_camp, jeonnam_review, jeonnam_tour, jeonnam_rtour)
seoul_review, seoul_tour, seoul_rtour = dele(seoul_camp, seoul_review, seoul_tour, seoul_rtour)
ulsan_review, ulsan_tour, ulsan_rtour = dele(ulsan_camp, ulsan_review, ulsan_tour, ulsan_rtour)
# 캠핑장 항목이 다른 데이터셋의 전처리 과정

In [279]:
korea_camp.to_csv('korea_camp2.csv')
korea_review.to_csv('korea_review2.csv')
korea_tour.to_csv('korea_tour2.csv')
korea_rtour.to_csv('korea_rtour2.csv')